#Lab.05 / IBM3202 – Modelamiento de Proteínas de Membrana usando Rosetta

##Aspectos Teóricos

Según Alford et al. 2020 [Biophys J 2020 118(8), 2042-2055], las proteínas de membrana colaboran con el entorno lipídico circundante para llevar a cabo procesos vitales esenciales. Constituyen el 30% de todas las proteínas y son objetivos de más del 60% de los productos farmacéuticos. Sin embargo, las dificultades experimentales han limitado nuestra comprensión de sus mecanismos moleculares de funcionamiento.


<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/mpmodelling_01.jpg'/>
<figcaption>FIGURA 1. El desafío de resolver la estructura de proteínas de membrana. Como se muestra en este gráfico, el número de proteínas de membrana resueltas antes del amplio uso de la criomicroscopía electrónica (cryoEM) era (y sigue siendo) significativamente inferior al número de proteínas solubles.

<br>Bijelic A & Rompel A (2015) <i>Coord Chem Rev 299, 22-38.
</i></figcaption></center>
</figure>

Por lo tanto, la modelización de proteínas de membrana es ideal para determinar la estructura de estas proteínas y para desarrollar nuevos tratamientos terapéuticos. En este tutorial, aprenderemos a modelar proteínas de membrana en PyRosetta y proporcionaremos ejemplos de los tipos de problemas que se pueden resolver con este marco de trabajo.


##Generalidades

En este tutorial, modelaremos diferentes homólogos con identidades de secuencia variables de la proteína de membrana Glicoforina A, también conocida como GYPA. Esta es una proteína formadora de poros que, en el caso del ortólogo humano, ha sido ampliamente estudiada mediante diferentes métodos de biología estructural, como la resonancia magnética nuclear (NMR) y la cristalografía de rayos X.

Esta visión general y tutorial incluyen información de este [tutorial](http://www.meilerlab.org/jobs/downloadfile/name/May2018_rosetta_cm_presentation.pdf) por Nica Marlow y Georg Kuenze.


**El Algoritmo**

En términos generales, el algoritmo consiste en una larga trayectoria de Monte Carlo que comienza desde una plantilla elegida al azar. La trayectoria de Monte Carlo utiliza los siguientes movimientos:

* Inserción de fragmentos en regiones no alineadas.

* Reemplazo de un segmento elegido al azar con el de una estructura de plantilla diferente.

* Minimización en el espacio cartesiano utilizando una versión suave (diferenciable) de la función de energía del centroide de Rosetta. Finalmente, esto se sigue de una optimización a nivel de átomos.

Consulta más en la [documentación oficial](https://www.rosettacommons.org/docs/latest/application_documentation/structure_prediction/RosettaCM).


**Diferencias entre Modelado con un Solo Molde y Modelado con Múltiples Moldes**

*Modelado con un Solo Molde:*
- Se utiliza un solo molde como entrada.
- Emplea fragmentos derivados de la secuencia y el molde.
- Se utiliza cuando los moldes disponibles tienen una identidad muy alta (>60%).

*Modelado con Múltiples Moldes:*
- Se utilizan múltiples moldes como entrada.
- Combina secciones de múltiples modelos enhebrados y fragmentos derivados de la secuencia.
- Se utiliza cuando los moldes disponibles tienen una baja identidad (30-50%).


**El protocolo**

El modelado comparativo de Rosetta (**RosettaCM**) realiza una serie compleja de pasos que se ilustran en el esquema a continuación, pero que se pueden resumir en tres pasos principales:

1. Alinear la secuencia objetivo con las secuencias de los moldes.

2. Realizar un enhebrado parcial de la secuencia objetivo en las estructuras de los moldes.

3. Combinar fragmentos de diferentes moldes utilizando la hibridación de RosettaCM.


<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/mpmodelling_02.jpg'/>
<figcaption>FIGURA 2. Un esquema simple del protocolo RosettaCM
<br>Song Y et al (2013) <i>Structure 21(10), 1735-1742</i></figcaption></center>
</figure>

**Archivos de Entrada para RosettaCM**

Mínimo indispensable:

* Estructuras enhebradas parcialmente (✅)

* Definición del movimiento y opciones (✅)

Específicos para proteínas de membrana (no necesarios si se modelan proteínas solubles):

* Regiones que atraviesan la membrana (archivo "span") (🙋)

* Parches de ponderación para la membrana (✅)

Archivos opcionales basados en la información disponible:

* Información de restricciones (por ejemplo, conectividad de pares de átomos)

* Conectividad de disulfuros


**Para una exploración profunda del modelado de proteínas de membrana con Rosetta, sugerimos revisar los siguientes artículos:**
1. Alford RF, Leman JK et al., “An Integrated Framework Advancing Membrane Protein Modeling and Design,” PLOS Computational Biology. 2015;11(9).
2. Alford RF, Fleming PJ, Fleming KG, Gray JJ (2020) “Protein structure prediction and design in a biologically-realistic implicit membrane” BioRxiv
3. Koehler Leman J, Muller BK, Gray JJ (2017) “Expanding the toolkit for membrane protein modeling in Rosetta” Bioinformatics 33(5):754-756.
4. Koehler Leman J & Gray JJ (2015) “Computational modeling of membrane proteins” Proteins Structure, Function, and Bioinformatics 83(1):1-24.

#Parte I – Explorando las secuencias de los ortólogos GYPA

1. Las secuencias con las que trabajaremos hoy corresponden a tres ortólogos de la proteína de membrana Glicoforina A. En nuestro caso particular, modelaremos los siguientes ortólogos utilizando PyRosetta:

  *   [Marmota flaviventris](https://www.ncbi.nlm.nih.gov/protein/XP_027782564.1?report=fasta&log$=seqview)

  *   [Gorilla gorilla](https://www.ncbi.nlm.nih.gov/protein/XP_007666620.2?report=fasta&log$=seqview)
  
  *   [Ornithorhynchus anatinus](https://www.uniprot.org/uniprot/O18798)

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/mpmodelling_03.png'/>
</figcaption></center>
</figure>

2. Por cuestiones de tiempo, proporcionaremos las secuencias ya alineadas y recortadas en un archivo en formato Grishin (otro formato de alineación), las estructuras enhebradas parcialmente necesarias para ejecutar el protocolo de modelado y los archivos de fragmentos:

   Encontrarás estos archivos dentro de su carpeta correspondiente en input_files. Si deseas generar estos archivos de entrada por ti mismo, consulta **Apéndice A**.


In [ ]:
#Descargamos los input desde Github
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/input_files/input_files_mp.zip
#Comando para eliminar en caso de que hayan archivos anteriores
!rm -rf input_files
#Descomprimir los archivos
!unzip input_files_mp.zip

#Parte II – Determinando las regiones que atraviesan membrana (span file) usando OCTOPUS server

Como mencionamos brevemente al principio de este tutorial (Visión General), necesitamos proporcionarle a Rosetta una manera de saber a qué residuos se debe aplicar un "potencial de membrana" específico e implícito. Estas instrucciones se generan en base al servidor de predicción de regiones transmembrana (TM) [**OCTOPUS**](https://octopus.cbr.su.se).


1. En [**OCTOPUS**](https://octopus.cbr.su.se), debes ingresar la secuencia de la proteína objetivo para la predicción de regiones transmembrana (TM). Puedes encontrar estas secuencias de proteínas dentro de la carpeta FASTA en los archivos de entrada descargados.


2. Luego, los archivos de topología deben procesarse para convertirlos en archivos "span", como se muestra en la celda de código a continuación:

In [ ]:
%%bash
cd /content/input_files/octopus
perl octopus2span.pl GYPA.topo.txt > GYPA.span
perl octopus2span.pl marmota.topo.txt > marmota.span
perl octopus2span.pl ornithorhynchus.topo.txt > ornithorhynchus.span
perl octopus2span.pl gorilla.topo.txt > gorilla.span

3. Una vez que hayas obtenido todos los archivos "span" de **OCTOPUS**, recuerda colocarlos dentro de la siguiente carpeta:


```
/content/input_files/octopus #Puedes abrir la carpeta aprentando esta dirección
```



# Parte III – Instalando PyRosetta y ejecutando los protocolos de modelamiento de proteínas de membrana

1. Comenzamos configurando PyRosetta en Google Colab, basándonos en la versión de PyRosetta que has compilado e instalado previamente y que está disponible en tu Google Drive.

**⚠️ADVERTENCIA:** ¡Recuerda que DEBES haber compilado e instalado PyRosetta antes de seguir este tutorial!


In [ ]:
!pip install pyrosettacolabsetup
import pyrosettacolabsetup
pyrosettacolabsetup.install_pyrosetta_on_google_drive()

2. En la siguiente celda de código, importamos los módulos de PyRosetta que vamos a utilizar:


In [ ]:
#Importamos los módulos de pyRosetta que utilizaremos
%%time
import pandas as pd
from pyrosetta import *
from pyrosetta.rosetta.protocols.rosetta_scripts import *
from pyrosetta import (
    init, pose_from_sequence, pose_from_file, Pose, MoveMap, create_score_function, get_fa_scorefxn,
    MonteCarlo, TrialMover, SwitchResidueTypeSetMover, PyJobDistributor,
)


3. A continuación, llevamos a cabo el protocolo de Modelado de Proteínas de Membrana. Esto se describe brevemente para el GYPA de _Gorilla_, mientras que el modelado del GYPA para los otros ortólogos es simplemente una modificación de este protocolo.


#### Gorilla 🦍

1. En la siguiente celda de código, proporcionamos a PyRosetta todas las **Flags** necesarias. Estas flags son equivalentes a instrucciones de línea de comandos para establecer la ruta de los archivos y configurar el valor de diversas opciones (numerosas). Las pasamos a PyRosetta, inicializándolo de esta manera.



```
pyrosetta.init(flags)
```



In [ ]:
#Las 'flags' son como las opciones del código en línea de comando
flags = f"""
-no_fconfig
-in:file:fasta ./input_files/fasta/gorillaGYPA.fasta
-out:prefix GYPA_gorilla_
-out:path:all ./output_files
-in:file:spanfile ./input_files/octopus/gorilla.span
-membrane:no_interpolate_Mpair
-membrane:Menv_penalties
-rg_reweight .1
-relax:fast
-relax:minimize_bond_angles
-relax:minimize_bond_lengths
-relax:jump_move true
-default_max_cycles 2
-relax:min_type lbfgs_armijo_nonmonotone
-score:weights input_files/wts/stage3_rlx_membrane.wts
-use_bicubic_interpolation
-hybridize:stage1_probability 1.0
-sog_upper_bound 15
-chemical:exclude_patches LowerDNA  UpperDNA Cterm_amidation SpecialRotamer VirtualBB ShoveBB VirtualDNAPhosphate VirtualNTerm CTermConnect sc_orbitals pro_hydroxylated_case1 pro_hydroxylated_case2 ser_phosphorylated thr_phosphorylated  tyr_phosphorylated tyr_sulfated lys_dimethylated lys_monomethylated  lys_trimethylated lys_acetylated glu_carboxylated cys_acetylated tyr_diiodinated N_acetylated C_methylamidated MethylatedProteinCterm
-analytic_etable_evaluation true
-linmem_ig 10
-out:mute core.io.pose_from_sfr.PoseFromSFRBuilder core.scoring.MembranePotential core.chemical.GlobalResidueTypeSet basic.io.database core.scoring.MembraneTopology core.scoring.CartesianBondedEnergy core.conformation.Conformation protocols.jd2.parser.ScoreFunctionLoader core.optimization.AtomTreeMinimizer core.pack
"""
pyrosetta.init(flags)

2. Uno de los conceptos cruciales en PyRosetta son las **poses**. Contienen la información, ya sea secuencia o estructura, a la cual se aplicará el protocolo.

   Teniendo esto en cuenta, aquí creamos un **objeto pose** a partir de una secuencia, que en este caso es la secuencia del GYPA de **_Gorilla_**. También especificamos un **res_type** que corresponde a **fullatom_standard**.


In [ ]:
#Aquí creamos un objeto que posea la secuencia
#también especificamos un res_type que corresponde a fullatom_standard
%%time
pose = pose_from_sequence('DNGEWIQLVPRFSGPEITLIIFGVMAGIIGTILLISYSIRRL', 'fa_standard')

3. Aunque hay múltiples formas de crear un protocolo en Rosetta, en nuestro caso vamos a crear un protocolo a partir de un **objeto XML**.

**NOTA:** ¡En la etiqueta **MOVERS** especificamos los **archivos de fragmentos** y el **archivo threaded_pdb**!


In [ ]:
#Aquí creamos un protocolo de Rosetta usando código XML
#Nota que en <MOVERS> especificamos los fragmentos y el PDB enhebrado!!

%%time
xml = pyrosetta.rosetta.protocols.rosetta_scripts.XmlObjects.create_from_string("""
<ROSETTASCRIPTS>
    <TASKOPERATIONS>
    </TASKOPERATIONS>
    <RESIDUE_SELECTORS>
    </RESIDUE_SELECTORS>
    <SCOREFXNS>
      <ScoreFunction name="stage1" weights="input_files/wts/stage1_membrane.wts" symmetric="0">
        <Reweight scoretype="atom_pair_constraint" weight="1"/>
      </ScoreFunction>
      <ScoreFunction name="stage2" weights="input_files/wts/stage2_membrane.wts" symmetric="0">
        <Reweight scoretype="atom_pair_constraint" weight="0.5"/>
      </ScoreFunction>
      <ScoreFunction name="fullatom" weights="input_files/wts/stage3_rlx_membrane.wts" symmetric="0">
        <Reweight scoretype="atom_pair_constraint" weight="0.5"/>
      </ScoreFunction>
      <ScoreFunction name="membrane" weights="membrane_highres_Menv_smooth" symmetric="0">
        <Reweight scoretype="cart_bonded" weight="0.5"/>
        <Reweight scoretype="pro_close" weight="0"/>
	 <Reweight scoretype="atom_pair_constraint" weight="0.5"/>
	</ScoreFunction>
    </SCOREFXNS>
    <FILTERS>
    </FILTERS>
    <MOVERS>
        	 <Hybridize name="hybridize" stage1_scorefxn="stage1" stage2_scorefxn="stage2" fa_scorefxn="fullatom" batch="1" stage1_increase_cycles="0.5" stage2_increase_cycles="0.5" linmin_only="1" add_hetatm="0" hetatm_to_protein_cst_weight="0"  >
       			<Fragments three_mers="/content/input_files/frags/gorillaGYPA.9mers" nine_mers="/content/input_files/frags/gorillaGYPA.9mers" />
	          	<Template pdb="/content/input_files/threaded_pdbs/5eh6_A.gorilla.pdb" cst_file="AUTO" weight="1.000" />
	       </Hybridize>
    </MOVERS>
    <APPLY_TO_POSE>
    </APPLY_TO_POSE>
    <PROTOCOLS>
      Add mover="add_memb"
	    <Add mover="hybridize"/>
    </PROTOCOLS>
<OUTPUT scorefxn="membrane"/>
</ROSETTASCRIPTS>
    """).get_mover("ParsedProtocol")

4. Una vez que nuestro protocolo está configurado, en las siguientes celdas de código:
   * Creamos una función de puntuación para membranas.
   * Luego, llamamos al PyJobDistributor y pasamos tres argumentos:
     * Un prefijo de nombre para guardar nuestros modelos como archivos PDB.
     * El número de modelos a generar.
     * La función de puntuación a emplear.
   * Pasamos nuestra pose al distribuidor de trabajos que acabamos de crear.
   * Después, creamos un DataFrame de Pandas para almacenar los resultados de las puntuaciones.
   * Finalmente, iteramos mientras los trabajos no estén completados.


In [ ]:
%%time
#Llamamos a la función de energía de membrana
membrane = pyrosetta.create_score_function("membrane_highres_Menv_smooth")
#Específicamos el prefijo, número de trabajos y la función de energía a usar
jd = pyrosetta.toolbox.py_jobdistributor.PyJobDistributor(pdb_name="GYPA_gorilla_",nstruct=2,scorefxn=membrane)
#Le entregamos la pose previamente creada al distribuidor de trabajos
jd.native_pose = pose
#Creamos un dataframe de panda donde se almacenarán los resultados
gdf = pd.DataFrame()
#Hacemos un bucle hasta que se completen todos los trabajos
while not jd.job_complete:
    test_pose = pose.clone()
    xml.apply(test_pose)
    test_gdf = pd.DataFrame.from_records(dict(test_pose.scores), index=[jd.current_name])
    gdf = gdf.append(test_gdf)
    jd.output_decoy(test_pose)

#### Ornitorrinchus 🥚

In [ ]:
flags = f"""
-no_fconfig
-in:file:fasta ./input_files/fasta/ornithorhynchusGYPA.fasta
-out:prefix GYPA_ornithorhynchus_
-in:file:spanfile ./input_files/octopus/ornithorhynchus.span
-membrane:no_interpolate_Mpair
-membrane:Menv_penalties
-rg_reweight .1
-relax:fast
-relax:minimize_bond_angles
-relax:minimize_bond_lengths
-relax:jump_move true
-default_max_cycles 2
-relax:min_type lbfgs_armijo_nonmonotone
-score:weights input_files/wts/stage3_rlx_membrane.wts
-use_bicubic_interpolation
-hybridize:stage1_probability 1.0
-sog_upper_bound 15
-chemical:exclude_patches LowerDNA  UpperDNA Cterm_amidation SpecialRotamer VirtualBB ShoveBB VirtualDNAPhosphate VirtualNTerm CTermConnect sc_orbitals pro_hydroxylated_case1 pro_hydroxylated_case2 ser_phosphorylated thr_phosphorylated  tyr_phosphorylated tyr_sulfated lys_dimethylated lys_monomethylated  lys_trimethylated lys_acetylated glu_carboxylated cys_acetylated tyr_diiodinated N_acetylated C_methylamidated MethylatedProteinCterm
-analytic_etable_evaluation true
-linmem_ig 10
-out:mute core.io.pose_from_sfr.PoseFromSFRBuilder core.scoring.MembranePotential core.chemical.GlobalResidueTypeSet basic.io.database core.scoring.MembraneTopology core.scoring.CartesianBondedEnergy core.conformation.Conformation protocols.jd2.parser.ScoreFunctionLoader core.optimization.AtomTreeMinimizer core.pack
"""
pyrosetta.init(flags)

In [ ]:
%%time
pose = pose_from_sequence('IDPEIEHIFTGPVAAVIIYAVVCGVIGTILFIALVIKVV', 'fa_standard')

In [ ]:
%%time
xml = pyrosetta.rosetta.protocols.rosetta_scripts.XmlObjects.create_from_string("""
<ROSETTASCRIPTS>
    <TASKOPERATIONS>
    </TASKOPERATIONS>
    <RESIDUE_SELECTORS>
    </RESIDUE_SELECTORS>
    <SCOREFXNS>
      <ScoreFunction name="stage1" weights="input_files/wts/stage1_membrane.wts" symmetric="0">
        <Reweight scoretype="atom_pair_constraint" weight="1"/>
      </ScoreFunction>
      <ScoreFunction name="stage2" weights="input_files/wts/stage2_membrane.wts" symmetric="0">
        <Reweight scoretype="atom_pair_constraint" weight="0.5"/>
      </ScoreFunction>
      <ScoreFunction name="fullatom" weights="input_files/wts/stage3_rlx_membrane.wts" symmetric="0">
        <Reweight scoretype="atom_pair_constraint" weight="0.5"/>
      </ScoreFunction>
      <ScoreFunction name="membrane" weights="membrane_highres_Menv_smooth" symmetric="0">
        <Reweight scoretype="cart_bonded" weight="0.5"/>
        <Reweight scoretype="pro_close" weight="0"/>
	 <Reweight scoretype="atom_pair_constraint" weight="0.5"/>
	</ScoreFunction>
    </SCOREFXNS>
    <FILTERS>
    </FILTERS>
    <MOVERS>
          AddMembraneMover name="add_memb"
        	 <Hybridize name="hybridize" stage1_scorefxn="stage1" stage2_scorefxn="stage2" fa_scorefxn="fullatom" batch="1" stage1_increase_cycles="0.5" stage2_increase_cycles="0.5" linmin_only="1" add_hetatm="0" hetatm_to_protein_cst_weight="0"  >
       			<Fragments three_mers="/content/input_files/frags/ornithorhynchusGYPA.3mers" nine_mers="/content/input_files/frags/ornithorhynchusGYPA.9mers" />
	          	<Template pdb="/content/input_files/threaded_pdbs/5eh6_A.ornithorhynchus.pdb" cst_file="AUTO" weight="1.000" />
	       </Hybridize>
    </MOVERS>
    <APPLY_TO_POSE>
    </APPLY_TO_POSE>
    <PROTOCOLS>
      Add mover="add_memb"
	    <Add mover="hybridize"/>
    </PROTOCOLS>
<OUTPUT scorefxn="membrane"/>
</ROSETTASCRIPTS>
    """).get_mover("ParsedProtocol")

In [ ]:
%%time
membrane = pyrosetta.create_score_function("membrane_highres_Menv_smooth")
jd = pyrosetta.toolbox.py_jobdistributor.PyJobDistributor(pdb_name="GYPA_ornitorrinchus",nstruct=2,scorefxn=membrane)
jd.native_pose = pose
odf = pd.DataFrame()
while not jd.job_complete:
    test_pose = pose.clone()
    xml.apply(test_pose)
    test_odf = pd.DataFrame.from_records(dict(test_pose.scores), index=[jd.current_name])
    odf = odf.append(test_odf)
    jd.output_decoy(test_pose)

####Marmota 🦡

In [ ]:
%%time
flags = f"""
-no_fconfig
-in:file:fasta ./input_files/fasta/marmotaGYPA.fasta
-out:prefix GYPA_marmota_
-out:path:all ./output_files
-in:file:spanfile ./input_files/octopus/marmota.span
-membrane:no_interpolate_Mpair
-membrane:Menv_penalties
-rg_reweight .1
-relax:fast
-relax:minimize_bond_angles
-relax:minimize_bond_lengths
-relax:jump_move true
-default_max_cycles 2
-relax:min_type lbfgs_armijo_nonmonotone
-score:weights input_files/wts/stage3_rlx_membrane.wts
-use_bicubic_interpolation
-hybridize:stage1_probability 1.0
-sog_upper_bound 15
-chemical:exclude_patches LowerDNA  UpperDNA Cterm_amidation SpecialRotamer VirtualBB ShoveBB VirtualDNAPhosphate VirtualNTerm CTermConnect sc_orbitals pro_hydroxylated_case1 pro_hydroxylated_case2 ser_phosphorylated thr_phosphorylated  tyr_phosphorylated tyr_sulfated lys_dimethylated lys_monomethylated  lys_trimethylated lys_acetylated glu_carboxylated cys_acetylated tyr_diiodinated N_acetylated C_methylamidated MethylatedProteinCterm
-analytic_etable_evaluation true
-linmem_ig 10
-out:mute core.io.pose_from_sfr.PoseFromSFRBuilder core.scoring.MembranePotential core.chemical.GlobalResidueTypeSet basic.io.database core.scoring.MembraneTopology core.scoring.CartesianBondedEnergy core.conformation.Conformation protocols.jd2.parser.ScoreFunctionLoader core.optimization.AtomTreeMinimizer core.pack
"""
pyrosetta.init(flags)

In [ ]:
%%time
pose = pose_from_sequence('KQGRNQIIHPFSEPVIILIIFAVMFGIIGTILLISFCVRRL', 'fa_standard')

In [ ]:
%%time
xml = pyrosetta.rosetta.protocols.rosetta_scripts.XmlObjects.create_from_string("""
<ROSETTASCRIPTS>
    <TASKOPERATIONS>
    </TASKOPERATIONS>
    <RESIDUE_SELECTORS>
    </RESIDUE_SELECTORS>
    <SCOREFXNS>
      <ScoreFunction name="stage1" weights="input_files/wts/stage1_membrane.wts" symmetric="0">
        <Reweight scoretype="atom_pair_constraint" weight="1"/>
      </ScoreFunction>
      <ScoreFunction name="stage2" weights="input_files/wts/stage2_membrane.wts" symmetric="0">
        <Reweight scoretype="atom_pair_constraint" weight="0.5"/>
      </ScoreFunction>
      <ScoreFunction name="fullatom" weights="input_files/wts/stage3_rlx_membrane.wts" symmetric="0">
        <Reweight scoretype="atom_pair_constraint" weight="0.5"/>
      </ScoreFunction>
      <ScoreFunction name="membrane" weights="membrane_highres_Menv_smooth" symmetric="0">
        <Reweight scoretype="cart_bonded" weight="0.5"/>
        <Reweight scoretype="pro_close" weight="0"/>
	 <Reweight scoretype="atom_pair_constraint" weight="0.5"/>
	</ScoreFunction>
    </SCOREFXNS>
    <FILTERS>
    </FILTERS>
    <MOVERS>
          AddMembraneMover name="add_memb"
        	 <Hybridize name="hybridize" stage1_scorefxn="stage1" stage2_scorefxn="stage2" fa_scorefxn="fullatom" batch="1" stage1_increase_cycles="0.5" stage2_increase_cycles="0.5" linmin_only="1" add_hetatm="0" hetatm_to_protein_cst_weight="0"  >
       			<Fragments three_mers="./input_files/frags/marmotaGYPA.3mers" nine_mers="./input_files/frags/marmotaGYPA.9mers" />
	          	<Template pdb="./input_files/threaded_pdbs/5eh6_A.marmota.pdb" cst_file="AUTO" weight="1.000" />
	       </Hybridize>
    </MOVERS>
    <APPLY_TO_POSE>
    </APPLY_TO_POSE>
    <PROTOCOLS>
      Add mover="add_memb"
	    <Add mover="hybridize"/>
    </PROTOCOLS>
<OUTPUT scorefxn="membrane"/>
</ROSETTASCRIPTS>
    """).get_mover("ParsedProtocol")

In [ ]:
membrane = pyrosetta.create_score_function("membrane_highres_Menv_smooth")
jd = pyrosetta.toolbox.py_jobdistributor.PyJobDistributor(pdb_name="GYPA_marmota_",nstruct=2,scorefxn=membrane)
jd.native_pose = pose
mdf = pd.DataFrame()
while not jd.job_complete:
    test_pose = pose.clone()
    xml.apply(test_pose)
    test_mdf = pd.DataFrame.from_records(dict(test_pose.scores), index=[jd.current_name])
    mdf = mdf.append(test_mdf)
    jd.output_decoy(test_pose)

####Human 🧑

In [ ]:
%%time
flags = f"""
-no_fconfig
-in:file:fasta ./input_files/fasta/GYPA.fasta
-parser:protocol ./input_files/cm.xml
-out:prefix GYPA_
-out:path:all ./output_files
-in:file:spanfile ./input_files/octopus/GYPA.span
-membrane:no_interpolate_Mpair
-membrane:Menv_penalties
-rg_reweight .1
-relax:fast
-relax:minimize_bond_angles
-relax:minimize_bond_lengths
-relax:jump_move true
-default_max_cycles 2
-relax:min_type lbfgs_armijo_nonmonotone
-score:weights input_files/wts/stage3_rlx_membrane.wts
-use_bicubic_interpolation
-hybridize:stage1_probability 1.0
-sog_upper_bound 15
-chemical:exclude_patches LowerDNA  UpperDNA Cterm_amidation SpecialRotamer VirtualBB ShoveBB VirtualDNAPhosphate VirtualNTerm CTermConnect sc_orbitals pro_hydroxylated_case1 pro_hydroxylated_case2 ser_phosphorylated thr_phosphorylated  tyr_phosphorylated tyr_sulfated lys_dimethylated lys_monomethylated  lys_trimethylated lys_acetylated glu_carboxylated cys_acetylated tyr_diiodinated N_acetylated C_methylamidated MethylatedProteinCterm
-analytic_etable_evaluation true
-linmem_ig 10
-out:mute core.io.pose_from_sfr.PoseFromSFRBuilder core.scoring.MembranePotential core.chemical.GlobalResidueTypeSet basic.io.database core.scoring.MembraneTopology core.scoring.CartesianBondedEnergy core.conformation.Conformation protocols.jd2.parser.ScoreFunctionLoader core.optimization.AtomTreeMinimizer core.pack
"""
pyrosetta.init(flags)

In [ ]:
%%time
pose = pose_from_sequence('ETGERVQLAHHFSEPEITLIIFGVMAGVIGTILLISYGIRRL', 'fa_standard')

In [ ]:
%%time
xml = pyrosetta.rosetta.protocols.rosetta_scripts.XmlObjects.create_from_string("""
<ROSETTASCRIPTS>
    <TASKOPERATIONS>
    </TASKOPERATIONS>
    <RESIDUE_SELECTORS>
    </RESIDUE_SELECTORS>
    <SCOREFXNS>
      <ScoreFunction name="stage1" weights="input_files/wts/stage1_membrane.wts" symmetric="0">
        <Reweight scoretype="atom_pair_constraint" weight="1"/>
      </ScoreFunction>
      <ScoreFunction name="stage2" weights="input_files/wts/stage2_membrane.wts" symmetric="0">
        <Reweight scoretype="atom_pair_constraint" weight="0.5"/>
      </ScoreFunction>
      <ScoreFunction name="fullatom" weights="input_files/wts/stage3_rlx_membrane.wts" symmetric="0">
        <Reweight scoretype="atom_pair_constraint" weight="0.5"/>
      </ScoreFunction>
      <ScoreFunction name="membrane" weights="membrane_highres_Menv_smooth" symmetric="0">
        <Reweight scoretype="cart_bonded" weight="0.5"/>
        <Reweight scoretype="pro_close" weight="0"/>
	 <Reweight scoretype="atom_pair_constraint" weight="0.5"/>
	</ScoreFunction>
    </SCOREFXNS>
    <FILTERS>
    </FILTERS>
    <MOVERS>
          AddMembraneMover name="add_memb"
        	 <Hybridize name="hybridize" stage1_scorefxn="stage1" stage2_scorefxn="stage2" fa_scorefxn="fullatom" batch="1" stage1_increase_cycles="0.5" stage2_increase_cycles="0.5" linmin_only="1" add_hetatm="0" hetatm_to_protein_cst_weight="0"  >
       			<Fragments three_mers="./input_files/frags/GYPA.3mers" nine_mers="./input_files/frags/GYPA.9mers" />
	          	<Template pdb="/content/input_files/threaded_pdbs/5eh6_A.human.pdb" cst_file="AUTO" weight="1.000" />
	       </Hybridize>
    </MOVERS>
    <APPLY_TO_POSE>
    </APPLY_TO_POSE>
    <PROTOCOLS>
      Add mover="add_memb"
	    <Add mover="hybridize"/>
    </PROTOCOLS>
<OUTPUT scorefxn="membrane"/>
</ROSETTASCRIPTS>
    """).get_mover("ParsedProtocol")

In [ ]:
%%time
membrane = pyrosetta.create_score_function("membrane_highres_Menv_smooth")
jd = pyrosetta.toolbox.py_jobdistributor.PyJobDistributor(pdb_name="GYPA_human_",nstruct=2,scorefxn=membrane)
jd.native_pose = pose
#Name of the score dataframe
hdf = pd.DataFrame()
while not jd.job_complete:
    test_pose = pose.clone()
    xml.apply(test_pose)
    test_hdf = pd.DataFrame.from_records(dict(test_pose.scores), index=[jd.current_name])
    hdf = hdf.append(test_hdf)
    jd.output_decoy(test_pose)

#Parte IV – Analizar y visualizar las estructuras modeladas

1. Ahora, necesitamos determinar nuestros mejores modelos analizando sus DataFrames de puntuaciones. Para esto, vamos a obtener el DataFrame de cada uno de los modelos que generamos con PyRosetta.

   Podemos inspeccionar visualmente estos DataFrames simplemente escribiendo sus nombres en un fragmento de código.


In [ ]:
#Invoca los diferentes dataframes que creamos
hdf


In [ ]:
odf


In [ ]:
mdf


In [ ]:
gdf

2. Dado que Rosetta utiliza un algoritmo estocástico para el modelado comparativo, solo estamos interesados en las estructuras de baja energía. Por lo tanto, podemos ordenar nuestros modelos por la columna total_score como se muestra en la siguiente celda de código:


In [ ]:
#hdf.sort_values('total_score')
gdf.sort_values('total_score')

3. Tener demasiadas tablas o tablas con demasiada información sería un desafío para analizar nuestros resultados. Para simplificar, podemos hacer un gráfico con el **total_score** de cada PDB para cada especie utilizando **matplotlib**.


In [ ]:
import matplotlib
import seaborn
frames = [hdf,odf,mdf,gdf]
results = pd.concat(frames)
matplotlib.rcParams['figure.figsize'] = [12.0, 8.0]
seaborn.barplot(x="total_score", y="index", data=results.sort_values('total_score').reset_index())

4. Finalmente, podemos echar un vistazo a nuestros mejores modelos utilizando **py3Dmol**.


In [ ]:
#!pip install py3Dmol
import py3Dmol
view=py3Dmol.view()
#Cargando el molde
view.addModel(open('5eh6.pdb', 'r').read(),'pdb')
view.addModel(open('GYPA_gorilla__0.pdb', 'r').read(),'pdb')
view.zoomTo()
view.setBackgroundColor('white')
view.setStyle({'chain':'A'},{'cartoon': {'color':'purple'}})
view.setStyle({'chain':' '},{'cartoon': {'color':'yellow'}})
view.show()
input()
view.png()

**📚TAREA:** Como desafío, utilizando lo que has aprendido en los tutoriales anteriores para superponer estructuras y determinar su similitud mediante el RMSD, toma tus mejores estructuras y compáralas para responder a la siguiente pregunta: **¿Es diferente la región transmembrana (TM) entre los ortólogos?**


5. Para finalizar, puedes descargar tus archivos de salida según se indica en la celda de código a continuación.


In [ ]:
#descarga los archivos finales
%%bash
zip -r example.zip ./outs
mkdir outs
cp *.pdb ./outs/

**Este es el final del quito tutorial. Buena ciencia!**

#Apéndice A

## Generando los archivos de entrada utilizando la versión independiente de línea de comandos de Rosetta**


Primero, necesitas obtener una licencia de Rosetta [AQUÍ](https://els2.comotion.uw.edu/product/rosetta), al igual que hiciste para PyRosetta.

### Descarga y extrae Rosetta en Google Colab (ETA ~15min)

In [ ]:
!wget --user=Academic_User --password=Xry3x4 https://www.rosettacommons.org/downloads/academic/3.12/rosetta_bin_linux_3.12_bundle.tgz
!tar xvf /content/rosetta_bin_linux_3.12_bundle.tgz

In [ ]:
!unzip input_files.zip

### Gerenar un archivo Grishin

El formato de archivo de alineación Grishin está documentado [aquí](https://www.rosettacommons.org/docs/latest/rosetta_basics/file_types/Grishan-format-alignment). A continuación, se muestra un ejemplo de su estructura:

```
## target_name template_name.pdb
#
scores from program: 0
0 HERE_GOES_YOUR_TARGET_SEQUENCE_ALIGNMENT_IN_ONE_LINE
0 HERE_GOES_YOUR_TEMPLATE_SEQUENCE_ALIGNMENT_IN_ONE_LINE
```
Este es uno de los archivos Grishin reales usados en el tutorial
```
## gorillaGYPA 5eh6_A.pdb
#
scores from program: 0
0 DNGEWIQLVPRFSGPEITLIIFGVMAGIIGTILLISYSIRRL
0 --------------PEITLIIFGVIAGVIGTILLISYGIRRL
```



In [ ]:
#Modifica esta celda para hacer un archivo Grishin a partir de un alineamiento clustal
%%bash
echo \
"""\
## gorillaGYPA 5eh6_A.pdb
#
scores from program: 0
0 DNGEWIQLVPRFSGPEITLIIFGVMAGIIGTILLISYSIRRL
0 --------------PEITLIIFGVIAGVIGTILLISYGIRRL
""" > /content/input_files/grishin/5eh6_on_gorillaGYPA.grishin

echo \
"""\
## GYPA 5eh6_A.pdb
#
scores from program: 0
0 ETGERVQLAHHFSEPEITLIIFGVMAGVIGTILLISYGIRRL
0 --------------PEITLIIFGVIAGVIGTILLISYGIRRL
""" > /content/input_files/grishin/5eh6_on_humanGYPA.grishin

echo \
"""\
## marmotaGYPA 5eh6_A.pdb
#
scores from program: 0
0 -KQGRNQIIHPFSEPVIILIIFAVMFGIIGTILLISFCVRRL
0 --------------PEITLIIFGVIAGVIGTILLISYGIRRL
""" > /content/input_files/grishin/5eh6_on_marmotaGYPA.grishin


echo \
"""\
## OrnithorhynchusGYPA 5eh6_A.pdb
#
scores from program: 0
0 ---IDPEIEHIFTGPVAAVIIYAVVCGVIGTILFIALVIKVV
0 --------------PEITLIIFGVIAGVIGTILLISYGIRRL
""" > /content/input_files/grishin/5eh6_on_ornithorhynchusGYPA.grishin

### Enhebrado parcial

In [ ]:
import urllib.request
urllib.request.urlretrieve('http://files.rcsb.org/download/5eh6.pdb', '/content/input_files/pdbs/5eh6.pdb')

In [ ]:
%%bash
cd /content/input_files/pdbs
/content/rosetta_bin_linux_2020.08.61146_bundle/tools/protein_tools/scripts/clean_pdb.py 5eh6.pdb A
mv 5eh6_A.* ../cleaned

In [ ]:
!/content/rosetta_bin_linux_2020.08.61146_bundle/main/source/bin/partial_thread.static.linuxgccrelease -in:file:fasta ./fasta/GYPA.fasta -in:file:alignment ./grishin/5eh6.grishin -in:file:template_pdb ./cleaned/5eh6_A.pdb
!mv 5eh6_A.pdb.pdb 5eh6_A.human.pdb
!/content/rosetta_bin_linux_2020.08.61146_bundle/main/source/bin/partial_thread.static.linuxgccrelease -in:file:fasta ./fasta/ornithorhynchusGYPA.fasta -in:file:alignment ./grishin/5eh6_on_ornithorhynchusGYPA.grishin -in:file:template_pdb ./cleaned/5eh6_A.pdb
!mv 5eh6_A.pdb.pdb 5eh6_A.ornithorhynchus.pdb
!/content/rosetta_bin_linux_2020.08.61146_bundle/main/source/bin/partial_thread.static.linuxgccrelease -in:file:fasta ./fasta/marmotaGYPA.fasta -in:file:alignment ./grishin/5eh6_on_marmotaGYPA.grishin -in:file:template_pdb ./cleaned/5eh6_A.pdb
!mv 5eh6_A.pdb.pdb 5eh6_A.gorilla.pdb
!/content/rosetta_bin_linux_2020.08.61146_bundle/main/source/bin/partial_thread.static.linuxgccrelease -in:file:fasta ./fasta/gorillaGYPA.fasta -in:file:alignment ./grishin/5eh6_on_gorillaGYPA.grishin -in:file:template_pdb ./cleaned/5eh6_A.pdb
!mv 5eh6_A.pdb.pdb 5eh6_A.marmota.pdb
!mv 5eh6_A.*pdb ./input_files/threaded_pdbs/

### Obteniendo fragmentos desde oldRobetta server


Según la documentación de Rosetta, los fragmentos son:

> (...) utilizados en el ensamblaje de proteínas, ya sea para la predicción de la estructura o el diseño, para reducir el tamaño del espacio de búsqueda para el plegamiento de proteínas. Son una parte central del diseño de Rosetta.

Para el propósito de este tutorial, hemos omitido esta parte. Sin embargo, en la siguiente figura puedes ver una captura de pantalla del [servidor Robetta](http://old.robetta.org/fragmentsubmit.jsp) donde puedes obtener los fragmentos. Para ello, debes registrarte y enviar una secuencia FASTA de tus proteínas objetivo. Ten en cuenta que la longitud de tu objetivo afectará el tiempo necesario para el proceso de selección de fragmentos.


<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/mpmodelling_04.png'/>
</figcaption></center>
</figure>

Cuando tu trabajo haya terminado, recibirás un correo electrónico con los fragmentos correspondientes. Debes descargarlos y colocarlos dentro de la carpeta "frags", asegurándote de que el nombre de archivo coincida con el especificado en la celda de RosettaScripts XML.
